In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import dask
import dask.array as da
import matplotlib.pyplot as plt
import tifffile
from aicsimageio import AICSImage
from cellpose import models
from skimage import color
from skimage.measure import regionprops_table
import napari

## User input

In [ ]:
# Enter the path to the .lif file to be processed
fn = r'Z:\zmbstaff\9309\Raw_Data\241220_ratio_cytoplasm_nucleus\STATeLight.lif'

# Enter the path to where the masks should be exported
export_dir_masks = r'Z:\zmbstaff\9309\Raw_Data\241220_ratio_cytoplasm_nucleus\STATeLight\roiImages'

# Enter the path to where the measurements should be exported
export_dir_measurements = r'Z:\zmbstaff\9309\Raw_Data\241220_ratio_cytoplasm_nucleus\STATeLight'

In [ ]:
nuclear_channel = 0
cytoplasm_channel = 1
channels_to_measure = [1,2]

## IO

In [ ]:
# load .lif for inspection:
lif_img = AICSImage(fn, reconstruct_mosaic=False)

In [ ]:
all_scenes = lif_img.scenes
all_scenes

In [ ]:
def parse_scenes(scenes: list[str]) -> pd.DataFrame:
    # Iterate through data and extract information
    indices = []
    names = []
    for i, path in enumerate(scenes):
        if ('control' not in path):
            indices.append(i)
            names.append(path)
    
    if len(names) != len(set(names)):
        raise ValueError('Scene names are not unique!')
    
    return indices, names

In [ ]:
scene_indices, scene_names = parse_scenes(all_scenes)
scene_dict = {name: indiex for name, indiex in zip(scene_names, scene_indices)}
scene_dict

In [ ]:
@dask.delayed
def lazy_load_dask_scene(lif_img, scene_index):
    lif_img.set_scene(scene_index)
    return lif_img.dask_data

def lazy_load_scenes(lif_img, scene_indices):
    lazy_list = [lazy_load_dask_scene(lif_img, i) for i in scene_indices]
    dask_list = dask.compute(*lazy_list)
    return da.stack(dask_list)

In [ ]:
scenes_data = lazy_load_scenes(lif_img, scene_indices)
scenes_data = scenes_data[:,0,:,0]
scenes_data = scenes_data.compute()
scenes_data.shape

The Axes are:
- 0: image no.
- 1: channel
- 2: y
- 3: x

In [ ]:
# load pixel sizes
# assume pixels sizes are the same for all scenes
dx = lif_img.physical_pixel_sizes.X
dy = lif_img.physical_pixel_sizes.Y

In [ ]:
plt.imshow(scenes_data[0,0])

## cellpose dask

In [ ]:
# assemble data for cellpose:
# c0: nuclear signal
# c1: cytoplasmic signal
data_cp = np.stack(
    [
        scenes_data[:,nuclear_channel],
        scenes_data[:,cytoplasm_channel]
    ],
    axis=1
)

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].imshow(data_cp[0,0])
axs[1].imshow(data_cp[0,1])

In [ ]:
def run_cellpose(block, cp_model_type, cp_channels, cp_diameter):
    data_cp = [scene for scene in block]

    model = models.Cellpose(gpu=True, model_type=cp_model_type)
    masks, *_ = model.eval(
        data_cp,
        diameter=cp_diameter,
        channels=cp_channels,
        flow_threshold=0.4,
        cellprob_threshold=0.0,
        do_3D=False,
        normalize=True,
    )

    return np.array(masks).astype(block.dtype)

In [ ]:
masks_nuc_cp = run_cellpose(
    data_cp[:,:1],
    cp_model_type='nuclei',
    cp_channels=[1,0],
    cp_diameter=50, # USER INPUT: approximate nucleus size in pixels
)

In [ ]:
masks_cell_cp = run_cellpose(
    data_cp,
    cp_model_type='cyto2',
    cp_channels=[2,1],
    cp_diameter=75, # USER INPUT: approximate nucleus size in pixels
)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(scenes_data, channel_axis = 1)
viewer.add_labels(masks_nuc_cp)
viewer.add_labels(masks_cell_cp)

# Measurements

In [ ]:
masks_cyto = []
masks_nuc = []
for i, (mask_nuc_cp, mask_cell_cp) in enumerate(zip(masks_nuc_cp, masks_cell_cp)):
    mask_cyto = mask_cell_cp.copy()
    mask_nuc = mask_cell_cp.copy()
    mask_cyto[mask_nuc_cp!=0] = 0
    mask_nuc[mask_cyto!=0] = 0
    masks_cyto.append(mask_cyto)
    masks_nuc.append(mask_nuc)
masks_cyto = np.array(masks_cyto)
masks_nuc = np.array(masks_nuc)

In [ ]:
def _measure_area(mask, spacing, suffix):
    # measure area
    props = pd.DataFrame(
        regionprops_table(
            mask,
            properties=('label', 'area', 'num_pixels'),
            spacing=spacing,
        )
    )
    props = props.rename(columns={
        'area':'area_'+suffix,
        'num_pixels': 'num_pixels_'+suffix,
    })
    props = props.set_index('label')
    return props

In [ ]:
def _measure_intensity(mask, intensity_image, spacing, channel_no, suffix):
    props = pd.DataFrame(
        regionprops_table(
            mask,
            intensity_image=intensity_image,
            properties=('label', 'intensity_mean','intensity_min','intensity_max', 'num_pixels'),
            spacing=spacing
        )
    )
    props['intensity_tot'] = props['intensity_mean'] * props['num_pixels']
    props = props.rename(
        columns={
            'intensity_mean':f'intensity_mean_c{channel_no}_{suffix}',
            'intensity_min':f'intensity_min_c{channel_no}_{suffix}',
            'intensity_max':f'intensity_max_c{channel_no}_{suffix}',
            'intensity_tot':f'intensity_tot_c{channel_no}_{suffix}',
        }
    )
    props = props.drop(columns='num_pixels')
    props = props.set_index('label')
    return props

In [ ]:
@dask.delayed
def _block_measurements(
        mask_cell,
        mask_cyto,
        mask_nuc,
        intensity_list,
        intensity_channels,
        spacing,
        scene_name,
):
    dy, dx = spacing

    # measure areas
    props_cell = _measure_area(mask=mask_cell, spacing=(dy,dx), suffix='cell')
    props_cyto = _measure_area(mask=mask_cyto, spacing=(dy,dx), suffix='cytoplasm')
    props_nuc = _measure_area(mask=mask_nuc, spacing=(dy,dx), suffix='nucleus')

    # area ratio nucleus: A(nuc) / A(cell)
    props_area_ratio = pd.DataFrame(props_nuc['area_nucleus'] / props_cell['area_cell'], columns=('area_ratio_nucleus',))

    # measure intensities
    props_int_cell_list = []
    props_int_cyto_list = []
    props_int_nuc_list = []
    for intensity_channel, intensity_image in zip(intensity_channels, intensity_list):
        props_int_cell_list.append(
            _measure_intensity(mask_cell, intensity_image, spacing=(dy,dx), channel_no=intensity_channel, suffix='cell')
        )
        props_int_cyto_list.append(
            _measure_intensity(mask_cyto, intensity_image, spacing=(dy,dx), channel_no=intensity_channel, suffix='cytoplasm')
        )
        props_int_nuc_list.append(
            _measure_intensity(mask_nuc, intensity_image, spacing=(dy,dx), channel_no=intensity_channel, suffix='nucleus')
        )
    
    # intensity ratio cytoplasm/nucleus Itot(cyto) / Itot(nuc)
    props_Itot_ratio_list = []
    for i, intensity_channel in enumerate(intensity_channels):
        itot_cyto = props_int_cyto_list[i][f'intensity_tot_c{intensity_channel}_cytoplasm']
        itot_nuc = props_int_nuc_list[i][f'intensity_tot_c{intensity_channel}_nucleus']
        props = pd.DataFrame(itot_cyto / itot_nuc, columns=(f'Itot(cyto)/Itot(nuc)_c{intensity_channel}',))
        props_Itot_ratio_list.append(props)

    # intensity ratio cytoplasm/nucleus Imean(cyto) / Imean(nuc)
    props_Imean_ratio_list = []
    for i, intensity_channel in enumerate(intensity_channels):
        imean_cyto = props_int_cyto_list[i][f'intensity_mean_c{intensity_channel}_cytoplasm']
        imean_nuc = props_int_nuc_list[i][f'intensity_mean_c{intensity_channel}_nucleus']
        props = pd.DataFrame(imean_cyto / imean_nuc, columns=(f'Imean(cyto)/Imean(nuc)_c{intensity_channel}',))
        props_Imean_ratio_list.append(props)

    # measure bounding box
    props_bbox = pd.DataFrame(
        regionprops_table(
            mask_cell,
            properties=('label', 'bbox'),
            spacing=(dy,dx)
        )
    )
    props_bbox = props_bbox.set_index('label')

    # merge all measurements
    props = pd.concat(
        [props_cell, props_cyto, props_nuc, props_area_ratio] + props_int_cell_list + props_int_cyto_list + props_int_nuc_list + props_Itot_ratio_list + props_Imean_ratio_list + [props_bbox,],
        axis=1,
    )
    props = props.reset_index()
    props.insert(0, 'scene', scene_name)

    props.loc[props['area_cytoplasm'].isna(), 'area_cytoplasm'] = 0
    props.loc[props['area_nucleus'].isna(), 'area_nucleus'] = 0

    return props

In [ ]:
meta = _block_measurements(
    mask_cell=masks_cell_cp[0],
    mask_cyto=masks_cyto[0],
    mask_nuc=masks_nuc[0],
    intensity_list=[scenes_data[0,c] for c in channels_to_measure],
    intensity_channels=channels_to_measure,
    spacing=(dy, dx),
    scene_name=scene_names[0],
).compute()

delayed_dfs = []
for mask_cell, mask_cyto, mask_nuc, intensity_list, scene_name in zip(
    masks_cell_cp,
    masks_cyto,
    masks_nuc,
    scenes_data[:,channels_to_measure],
    scene_names,
):
    delayed_dfs.append(
        _block_measurements(
            mask_cell,
            mask_cyto,
            mask_nuc,
            intensity_list,
            channels_to_measure,
            (dy, dx),
            scene_name,)
    )
#measurements = dd.from_delayed(delayed_dfs, meta=meta).compute()
measurements = pd.concat(dask.compute(*delayed_dfs), axis=0).reset_index(drop=True)
measurements.insert(1, 'timepoint', measurements['scene'].apply(lambda x: x.split('/')[0]))
measurements.insert(2, 'position', measurements['scene'].apply(lambda x: int(x.split('Position ')[1])))

In [ ]:
measurements

# Filter

In [ ]:
measurements_to_plot = [
    'area_cell',
    'area_cytoplasm',
    'area_nucleus',
    'area_ratio_nucleus',
    'intensity_mean_c1_cell',
    'intensity_mean_c1_cytoplasm',
    'intensity_mean_c1_nucleus',
    'Imean(cyto)/Imean(nuc)_c1',
]

fig, axs = plt.subplots(2,4, figsize=(16,6))

for ax, col in zip(axs.flatten(), measurements_to_plot):
    ax.set_title(col)
    ax.hist(measurements[col].to_numpy(), bins=64)
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

fig.suptitle('Unfiltered measurements', weight='bold')
plt.tight_layout()

In [ ]:
# USER INPUT REQUIRED:
# specify filters to use:
measurements_filtered = measurements.query(
    "100 < area_cell"
    " and "
    "0 < area_cytoplasm"
    " and "
    "50 < area_nucleus < 200"
    " and "
    "10 < intensity_mean_c1_cell"
)

# filter out cells touching the border
img_shape_y, img_shape_x = scenes_data.shape[-2:]
measurements_filtered = measurements_filtered[
    (measurements_filtered['bbox-0'] > 0) &
    (measurements_filtered['bbox-1'] > 0) &
    (measurements_filtered['bbox-2'] < img_shape_y) &
    (measurements_filtered['bbox-3'] < img_shape_x)
]

In [ ]:
measurements_filtered

In [ ]:
measurements_to_plot = [
    'area_cell',
    'area_cytoplasm',
    'area_nucleus',
    'area_ratio_nucleus',
    'intensity_mean_c1_cell',
    'intensity_mean_c1_cytoplasm',
    'intensity_mean_c1_nucleus',
    'Imean(cyto)/Imean(nuc)_c1',
    #'Itot(cyto)/Itot(nuc)_c1',
]

fig, axs = plt.subplots(2,4, figsize=(16,6))

for ax, col in zip(axs.flatten(), measurements_to_plot):
    ax.set_title(col)
    ax.hist(measurements_filtered[col].to_numpy(), bins=64)
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

fig.suptitle('Filtered measurements', weight='bold')
plt.tight_layout()

In [ ]:
masks_cell_cp_filtered = masks_cell_cp.copy()
masks_cyto_filtered = masks_cyto.copy()
masks_nuc_filtered = masks_nuc.copy()
for i, scene_name in enumerate(scene_names):
    labels_filtered = measurements_filtered.query("scene == @scene_name")['label'].unique()
    masks_cell_cp_filtered[i][~np.isin(masks_cell_cp_filtered[i], labels_filtered)] = 0
    masks_cyto_filtered[i][~np.isin(masks_cyto_filtered[i], labels_filtered)] = 0
    masks_nuc_filtered[i][~np.isin(masks_nuc_filtered[i], labels_filtered)] = 0

In [ ]:
viewer = napari.Viewer()
viewer.add_image(scenes_data[:,[nuclear_channel, cytoplasm_channel]], channel_axis=1)
viewer.add_labels(masks_nuc, name='masks_nuc_all', visible=False)
viewer.add_labels(masks_cell_cp, name='masks_cell_all', visible=False)
viewer.add_labels(masks_cell_cp_filtered, name='masks_cell_filt')

### Aggregate measurements

In [ ]:
measurements_filtered_aggregated = measurements_filtered.drop(
    columns=['scene', 'position', 'label','bbox-0', 'bbox-1', 'bbox-2', 'bbox-3']
).groupby('timepoint', sort=False).agg(['mean', 'std', 'sem', 'count'])

In [ ]:
measurements_filtered_aggregated

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))

for c in channels_to_measure:
    ax.errorbar(
        measurements_filtered_aggregated[f'Imean(cyto)/Imean(nuc)_c{c}'].index,
        measurements_filtered_aggregated[f'Imean(cyto)/Imean(nuc)_c{c}']['mean'],
        yerr=measurements_filtered_aggregated[f'Imean(cyto)/Imean(nuc)_c{c}']['sem'],
        fmt='o',
        capsize=5,
        label=f'Channel {c}',
        
    )
ax.set_ylabel(r'$I_{mean}(cytoplasm) / I_{mean}(nucleus)$')
ax.legend(loc='upper right')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))

for c in channels_to_measure:
    ax.errorbar(
        measurements_filtered_aggregated[f'Itot(cyto)/Itot(nuc)_c{c}'].index,
        measurements_filtered_aggregated[f'Itot(cyto)/Itot(nuc)_c{c}']['mean'],
        yerr=measurements_filtered_aggregated[f'Itot(cyto)/Itot(nuc)_c{c}']['sem'],
        fmt='o',
        capsize=5,
        label=f'Channel {c}',
        
    )
ax.set_ylabel(r'$I_{tot}(cytoplasm) / I_{tot}(nucleus)$')
ax.legend(loc='upper right')

# Export

In [ ]:
# create export directory:
os.makedirs(Path(export_dir_masks) / 'masks_cell', exist_ok=True)
os.makedirs(Path(export_dir_masks) / 'masks_cytoplasm', exist_ok=True)
os.makedirs(Path(export_dir_masks) / 'masks_nucleus', exist_ok=True)
os.makedirs(Path(export_dir_masks) / 'masks_cell_filtered', exist_ok=True)
os.makedirs(Path(export_dir_masks) / 'masks_cytoplasm_filtered', exist_ok=True)
os.makedirs(Path(export_dir_masks) / 'masks_nucleus_filtered', exist_ok=True)
os.makedirs(Path(export_dir_measurements), exist_ok=True)

In [ ]:
# export measurements
measurements.to_csv(os.path.join(export_dir_measurements, 'measurements_unfiltered.csv'), index=False)
measurements_filtered.to_csv(os.path.join(export_dir_measurements, 'measurements_filtered.csv'), index=False)
measurements_filtered_aggregated.to_csv(os.path.join(export_dir_measurements, 'measurements_filtered_summary.csv'), index=False)

In [ ]:
for scene_name, mask_cell, mask_cyto, mask_nuc, mask_cell_filt, mask_cyto_filt, mask_nuc_filt in zip(scene_names, masks_cell_cp, masks_cyto, masks_nuc, masks_cell_cp_filtered, masks_cyto_filtered, masks_nuc_filtered):
    scene_name_new = scene_name.replace('/','_')
    export_name = Path(export_dir_masks) / 'masks_cell' / f'{scene_name_new}_mask.tif'
    tifffile.imwrite(export_name, mask_cell, photometric='minisblack')
    export_name = Path(export_dir_masks) / 'masks_cytoplasm' / f'{scene_name_new}_mask.tif'
    tifffile.imwrite(export_name, mask_cyto, photometric='minisblack')
    export_name = Path(export_dir_masks) / 'masks_nucleus' / f'{scene_name_new}_mask.tif'
    tifffile.imwrite(export_name, mask_nuc, photometric='minisblack')
    export_name = Path(export_dir_masks) / 'masks_cell_filtered' / f'{scene_name_new}_mask.tif'
    tifffile.imwrite(export_name, mask_cell_filt, photometric='minisblack')
    export_name = Path(export_dir_masks) / 'masks_cytoplasm_filtered' / f'{scene_name_new}_mask.tif'
    tifffile.imwrite(export_name, mask_cyto_filt, photometric='minisblack')
    export_name = Path(export_dir_masks) / 'masks_nucleus_filtered' / f'{scene_name_new}_mask.tif'
    tifffile.imwrite(export_name, mask_nuc_filt, photometric='minisblack')